Let's sart by adding our packages

In [1]:
import pandas as pd
import sqlite3
import datetime

Now that that's done, let's assemle our dataframe

In [2]:
df = pd.read_csv('gamesales.csv')

df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


I'm gonna minify this down to just name, platform and year because it'll be easier. I'm also going to take this as a chance to cast the year to int

In [52]:
df_minned = df[['Name', 'Platform', 'Genre', 'Year', 'Publisher']].dropna()

#oops there are Nans here I think, so let's fix this by rounding NAN values to 1000

# df_no_nan = df_minned['Year'].fillna(0)

#This function will allow me to assign franchises

df_minned['Year'] = df_minned['Year'].astype('int')

df_minned = df_minned.rename(columns=\
{"Year": "year_published", "Platform": "console", "Name": "game_title", "Genre": 'genre', "Publisher": 'publisher'})

df_minned['tagblob'] = df_minned['game_title'] + ' ' + df_minned['console'] + ' ' + df_minned['genre'] + ' ' + df_minned['publisher']

df_minned['pub_date'] = datetime.datetime.now()

df_minned

,game_title,console,genre,year_published,publisher,tagblob,pub_date
0,Wii Sports,Wii,Sports,2006,Nintendo,Wii Sports Wii Sports Nintendo,2022-03-27 21:46:54.665946
1,Super Mario Bros.,NES,Platform,1985,Nintendo,Super Mario Bros. NES Platform Nintendo,2022-03-27 21:46:54.665946
2,Mario Kart Wii,Wii,Racing,2008,Nintendo,Mario Kart Wii Wii Racing Nintendo,2022-03-27 21:46:54.665946
3,Wii Sports Resort,Wii,Sports,2009,Nintendo,Wii Sports Resort Wii Sports Nintendo,2022-03-27 21:46:54.665946
4,Pokemon Red/Pokemon Blue,GB,Role-Playing,1996,Nintendo,Pokemon Red/Pokemon Blue GB Role-Playing Nintendo,2022-03-27 21:46:54.665946
...,...,...,...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA,Platform,2002,Kemco,Woody Woodpecker in Crazy Castle 5 GBA Platfor...,2022-03-27 21:46:54.665946
16594,Men in Black II: Alien Escape,GC,Shooter,2003,Infogrames,Men in Black II: Alien Escape GC Shooter Infog...,2022-03-27 21:46:54.665946
16595,SCORE International Baja 1000: The Official Game,PS2,Racing,2008,Activision,SCORE International Baja 1000: The Official Ga...,2022-03-27 21:46:54.665946
16596,Know How 2,DS,Puzzle,2010,7G//AMES,Know How 2 DS Puzzle 7G//AMES,2022-03-27 21:46:54.665946


In [53]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("../community/db.sqlite3")

cur = con.cursor()

# let's make sure this works
for row in cur.execute('SELECT * FROM polls_game;'):
    print(row)

# Be sure to close the connection
con.close()

Now let's chuck it in the databse

In [54]:
con = sqlite3.connect("../community/db.sqlite3")

df_minned.to_sql("polls_game", con, if_exists='append', index=False)

con.close()